# Tworzenie zasobów + wykrywanie encji nazwanych

Algorytmy wykorzystywane w problemach przetwarzania języka naturalnego opierają najczęściej swoje działanie o analizę dużych korpusów danych. O ile w zadaniach konkursowych często odpowiednie dane są już przygotowane, o tyle tworząc własne eksperymenty, często musimy sami pozyskać dane i przetransformować do użytecznej postaci.

Dzisiejsze laboratoria dotyczyć będą tworzenia korpusów danych, tworzenia reprezentacji CoNNL i wykorzystania jej do zadania wykrywania encji nazwanych.

## Automatyczne pozyskiwanie surowych danych tekstowych
Dotychczas omawiane metody działały na surowym tekście, który transformowany był do odpowiedniej reprezentacji wektorowej (Bag of words, bag of ngrams, embeddingi). Jak zautomatyzować pozyskiwanie takich surowych danych z internetu?

W tej części skupimy się na stworzeniu automatycznego pobieracza danych, który działać będzie w dwóch "obszarach":
<ol>
<li>crawler: moduł odwiedzający kolejne strony internetowy</li>
<li>scraper: moduł ekstrahujący treść z konkretnych stron internetowych</li>
</ol>

Wykorzystajmy do tego dwie biblioteki: 

**urllib** - do odwiedzania stron

**BeautifulSoup** - do parsowania danych (np. w formacie HTML).

## Zadanie1 (2pkt): Napisz prosty ekstraktor danych ze stron WWW odwiedzający kilka podstron
Ekstraktor ma odwiedzić zadaną stronę internetową, pobrać zawartość wszystkich tekstów wewnątrz paragrafów (wewnątrz tagów P zawartych w pobranym dokumencie HTML), a następnie odwiedzić 5 dowolnych linków z tej strony i z nich analogicznie pobrać zawartość.
Łącznie powinniśmy otrzymać dane z 6 adresów internetowch (strona główna + 5 linków ze strony głównej).

Do napisania crawlera przydać się mogą następujące funkcje:

urllib.request.urlopen() - do pobrania zawartości strony
findAll() na obiekcie BeautifulSoup, można ją wykorzystać do przeiterowania po wszystkich tagach danego rodzaju
get_text() - Istnieje duża szansa, że wewnątrz tagów P znajdą się również inne tagi HTML, chcielibyśmy oczyścić 
z nich tekst. Można to zrobić albo z wyrażeniami regularnymi (robiliśmy takie zadanie na pierwszych laboratoriach!), albo użyć właśnie funkcji get_text() z BeautifulSoup

Linki do dokumentacji:
urllib, pobieranie danych: https://docs.python.org/3/howto/urllib2.html
beautifulSoup: https://www.crummy.com/software/BeautifulSoup/bs4/doc/ (przeczytanie QuickStart jest wystarczające do zrobienia tego zadania)


In [1]:
import urllib.request
from bs4 import BeautifulSoup
import re

def find_addressess(website):
    with urllib.request.urlopen(website) as response:
        html_data = response.read()
    soup = BeautifulSoup(html_data, 'html.parser')
    links = soup.find_all('a', href = True)
    url_links = [link['href'] for link in links]
    max_i = min(5, len(url_links))
    sites = A=[[website + url_links[i], url_links[i]][url_links[i].startswith('http')] for i in range(max_i)]
    sites.append(website)
    return sites
       

def simple_p_tags_extractor(website):
    with urllib.request.urlopen(website) as response:
        html_data = response.read()
    soup = BeautifulSoup(html_data, 'html.parser')
    p_tags = soup.find_all('p')
    p_tags = [tag.get_text() for tag in p_tags]
    
    return p_tags
    
sites = find_addressess('https://www.crummy.com/')
for site in sites:
    p_tags = simple_p_tags_extractor(site)
    print('---------------------')
    print(site)
    print('---------------------')
    print(p_tags)
    print('---------------------\n')

---------------------
https://www.crummy.com/self/
---------------------
["Hi, I'm Leonard Richardson. I answer to Leonard or leonardr. I like\ndinosaurs, turtles, trilobites, and humans. This is my website. It's\nbeen around, in one form or another, since 1996.\n\nI was born in Los Angeles and raised in Arvin, California. Now I\nlive in New York City. I write programs and\nprose, including a weblog. My\nwonderful wife, Sumana\nHarihareswara, has a proportionately wonderful weblog.\n\nMy hobby is entertaining people. Thus, this web site. I have a lot\nof ideas, and I hope you find their fruits fun and useful.\n\nWho can tell what the future may bring? I may rise to conquer the\nworld, or my plans may be dashed by a rag-tag band of misfits. In any\nevent, it should be interesting.\nMy email address is leonardr@segfault.org. I don't\nget a whole lot of encrypted email, but maybe you should send me\nsome. Here's my GPG key.\n\nThis document (source) is part of Crummy, the webspace of Leon

# Zadanie 2 - CONLL
Dane ustrukturyzowane w formacie CONLL.

Niektóre algorytmy korzystają z dodatkowych metadanych opisujących poszczególne tokeny (słowa). Bardzo popularnym formatem zapisu takich danych jest format CONLL. 

Reprezentacja CONLL polega na tym, że dany tekst dzielony jest na zdania, a następnie każde zdanie dzielone jest na tokeny (tokenizowane). Następnie dla każdego tokenu tworzymy listę opisującą cechy tego tokenu (słowa).
Poniżej przykład wektora opisującego każdy token zadanego tekstu:
<ol>
    <li>ID - numer porządkowy tokenu w zdaniu</li>
    <li>text - tekst tokenu w formie nieprzetworzonej</li>
    <li>Part of Speech tag (POS tag) - informacja o części mowy, która powiązana jest z tym słowem </li>
    <li>is digit - flaga (o wartościach 0 lub 1), która informuje nas czy dany token jest liczbą</li>
    <li>is punct - flaga (o wartościach 0 lub 1), która informuje nas czy dany token jest znakiem interpunkcyjnym</li>
</ol>

Wektory cech dla kolejnych słów zapisywane są pod sobą. **Separatorem cech w wektorze jest pojedyncza spacja.**

**Zdania zwyczajowo oddzielamy od siebie podwójnym znakiem nowej linii.**

Historycznie CONLL był bardzo konkretnym formatem danych w którym mieliśmy z góry narzucone cechy (np. format CONLL-U https://universaldependencies.org/docs/format.html). Liczba cech ewoluowała jednak w czasie i w wielu miejscach CONLL stał się synonimem ogólnego formatu, w którym dobór cech zależy tylko od nas, jednak stałym jest zapis sekwencji tokenów jako sekwencji wierszy w tekście, gdzie każdy wiersz jest listą oddzielonych spacją wartości (cech), a zdania oddzielone są od siebie podwójnym znakiem nowej linii.


### Przykład:

Tekst: Kasia kupiła 2 lizaki: truskawkowy i zielony. Kasia używa Apple IPhone 5 i IPad.

Reprezentacja CONLL **(spacje separujące kolumny zostały zwielokrotnione na potrzeby zwiększenia czytelności)**
<pre>
1 Kasia  RZECZOWNIK 0 0
2 kupiła CZASOWNIK  0 0
3 2      LICZEBNIK  1 0
4 lizaki RZECZOWNIK 0 0
5 .      _          0 1

1 Kasia  RZECZOWNIK 0 0
2 używa  CZASOWNIK  0 0
3 Apple  RZECZOWNIK 0 0
4 IPhone RZECZOWNIK 0 0
5 5      LICZEBNIK  1 0
6 i      SPÓJNIK    0 0
7 iPad   RZECZOWNIK 0 0
8 .      _          0 1
</pre>

**Zadanie 2a (0.5 pkt)**: Napisz funkcję, która z zadanego tekstu w formie surowego tekstu stworzy reprezentację CONLL opisaną wcześniej wymienionymi atrybutami (ID, text, POS-tag, is_digit, is_punct).

Wykorzystaj sentence splitter i tokenizator z NLTK. Do uzyskania informacji o POS-tagach każdego tokenu wykorzystaj funkcję nltk.pos_tag(). W kolumnie związanej z POS-tagiem zapisz pos tag w takiej formie, w jakiej uzyskamy go z funkcji pos_tag (pos_tag() zwraca formy skrótowe, np. 'NN' dla rzeczowników), nie trzeba więc zamieniać napisu "NN" na "RZECZOWNIK".


In [2]:
import nltk
import string
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

def generate_conll(text):
    sentences = nltk.sent_tokenize(text)
    for sent in sentences:
        words = nltk.word_tokenize(sent)
        for i, tag in enumerate(nltk.pos_tag(words), 1):
            print(i, tag[0], tag[1], [0, 1][tag[0].isdigit()], [0, 1][tag[0] in string.punctuation])
        print('\n')
    pass


generate_conll("Kate uses IPhone 5 and IPad. Kate bought 2 lolipops.")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
1 Kate NNP 0 0
2 uses VBZ 0 0
3 IPhone NNP 0 0
4 5 CD 1 0
5 and CC 0 0
6 IPad NNP 0 0
7 . . 0 1


1 Kate NNP 0 0
2 bought VBD 0 0
3 2 CD 1 0
4 lolipops NNS 0 0
5 . . 0 1





Wyobraźmy sobie teraz, że chcielibyśmy wykrywać wzmianki o urządzeniach elektronicznych w tekście. W jaki sposób zakodować informację o (potencjalnie wielotokenowych) nazwach produktów w CONLL, tak, aby później móc wykonać proces uczenia?

Dodajmy w naszym CONLLu dodatkową kolumnę reprezentującą informację o urządzeniach elektronicznych.
Nazwy urządzeń mogą składać się potencjalnie z wielu słów.
Do zakodowania wielotokenowych tekstów używa się najczęściej notacji IOB, gdzie każda literka skrótu oznacza interpretację aktualnego słowa:
<ul>
    <li> B = begin, marker, który mówi, że aktualne słowo to początek nazwy </li>
    <li> I = inside, marker, który mówi, że aktualne słowo to kontynacja nazwy, która rozpoczyna się wystąpieniem wcześniejszego B</li>
    <li> O = outside, marker, który mówi, że aktualne słowo nie jest interesującą nas nazwą (jest poza nią) </li>
</ul>

Po dodaniu nowej kolumny (na końcu) nasz CONLL przybiera postać:

<pre>
1 Kasia  RZECZOWNIK 0 0 O
2 kupiła CZASOWNIK  0 0 O
3 2                 1 0 O
4 lizaki RZECZOWNIK 0 0 O
5 .      _          0 1 O

1 Kasia  RZECZOWNIK 0 0 O
2 używa             0 0 O
3 Apple  RZECZOWNIK 0 0 B
4 IPhone RZECZOWNIK 0 0 I
5 5                 1 0 I
6 i      SPÓJNIK    0 0 O
7 iPad   RZECZOWNIK 0 0 B
8 .      _          0 1 0
</pre>

Zwróćcie Państwo uwagę na ostatnią kolumnę, czytając tekst od góry w dół, wystąpienie literki "B" oznacza początek interesującej frazy (Apple), jeśli zaraz za "B" pojawia się sekwencja oznaczona jako "I" - kolejne tokeny stanowią kontynuację interesującej nas frazy, w tym przypadku 3 tokeny "Apple IPhone 5" tworzą jeden byt. Poza tym widzimy, że "iPad" stanowi osobny, jednotokenowy byt.

Po co rozróżniać pomiędzy "B", "I" i "O", czy nie można uwzględnić tylko dwóch tagów "wewnątrz frazy", "poza frazą"? Teoretycznie można, ale wprowadzimy w ten sposób sytuacje niejednoznaczne. 

Sprawdźmy to na przykładzie sekwencji "XBox Playstation" reprezentującej 2 osobne byty. Używając tagowania IOB nasza sekwencja wyglądałaby tak:

XBox B
PlayStation B

Widzimy więc, że dwa tagi "B" oznaczają dwa początki osobnych fraz. Co jednak gdybyśmy używali tagów "wewnątrz (interesującej nas) frazy", "poza (interesującą nas) frazą"?

XBox "wewnątrz (interesującej nas) frazy"
Playstation "wewnątrz (interesującej nas) frazy"

W tej sytuacji oznaczyliśmy poprawnie oba tokeny jako części interesujących nas fraz. Jednak nie wiemy, czy XBox Playstation to jedna, czy dwie osobne frazy (byty) -- stąd format IOB jest zdecydowanie bezpieczniejszym wyborem.

**Zadanie 2b (0.5 pkt)**: Napisz funkcję, która wygeneruje CONLL z uwzględnieniem tagów IOB dotyczących urządzeń.
Nasza funkcja posiada teraz dodatkowy argument devices, który zawiera listę obiektów, które opisują gdzie (przesunięcie znakowe) znajduje się początek i koniec wzmianek.


In [3]:
import string
import spacy
import nltk

def generate_CONLL(text, devices=[]):
    sentences = nltk.sent_tokenize(text)
    nlp = spacy.load("en_core_web_sm")
    docs = [nlp(sent) for sent in sentences]
    for doc in docs:
      for i, token in enumerate(doc, 1):
          iob_tag = 'O'
          for dic in devices:
            if token.idx == dic["begin"]:
                iob_tag = 'B'
            elif token.idx > dic["begin"] and token.idx < dic["end"]:
                iob_tag = 'I'
          print(i, token, token.pos_, [0, 1][token.is_digit], [0, 1][token.is_punct], iob_tag)
      print('\n')
    pass

# parametr devices to lista słowników w którym mamy informację o numerze znaku na którym fraza się zaczyna i kończy (zobacz: próba wywołania w ostatniej linijce) (litera I z Iphone występuje na 10 znaku)
# Zapoznaj się z dokumentacją SpaCy (obiekt Token), aby zobaczyć jak wydobyć informację o pozycji danego słowa w zdaniu/dokumencie.
    
generate_CONLL("Kate uses IPhone 5 and IPad. Kate bought 2 lolipops.", devices=[{"begin": 10, "end":18}, {"begin": 23, "end": 27}])

1 Kate PROPN 0 0 O
2 uses VERB 0 0 O
3 IPhone PROPN 0 0 B
4 5 NUM 1 0 I
5 and CCONJ 0 0 O
6 IPad PROPN 0 0 B
7 . PUNCT 0 1 O


1 Kate PROPN 0 0 O
2 bought VERB 0 0 O
3 2 NUM 1 0 I
4 lolipops NOUN 0 0 I
5 . PUNCT 0 1 O




Często chcemy w tekście naraz oznaczać byty, które należą do różnych kategorii, np. lokacje, numery telefonów, daty, wzmianki o osobach. W takich sytuacjach używa się również kodowania IOB jednak wzbogaca się etykiety o odpowiednie informacje używając formatu:

{tag IOB}-{etykieta kategorii}

Stąd daty przyjmują oznaczenia: B-DATE / I-DATE, osoby B-PERSON / I-PERSON, numery telefonów B-PHONENUMBER / I-PHONENUMBER, lokacje: B-LOCATION / I-LOCATION itp. Wiemy zatem czy dany token należy do interesującej nas frazy i do jakiej kategorii przypisana jest ta fraza.

## Wykrywanie encji nazwanych (Named Entity Recognition - NER)

Dotychczas na większości zajęć rozważaliśmy problem klasyfikacji, w którym całym dokumentom przypisywalśmy pojedynczą etykietę (sentyment związany z dokumentem, informacja o tym, czy tekst jest spamowy, etykieta mówiąca o tym w jakim języku napisany jest dokument). Warto jednak również wspomnieć o tzw. tagowaniu sekwencji, które dla każdego elementu sekwencji (słowa) nadaje odpowiednią etykietę.

Gdzie taka procedura ma zastosowanie? Wymieńmy kilka przykładów 
<ol>
    <li>Wykrywanie wyrażeń dotyczących miejsc, ludzi, czasu, lokalizacji itp. - każde kolejne słowo tagowane jest informacją mówiącą o tym, czy dane słowo jest częścią pożądanego przez nas typu (np. częścią lokalizacji), czy nie (np. z użyciem kodowania IOB, o którym mówiliśmy przy okazji CONLL)</li>
    <li>Tagowanie częściami mowy - każde słowo otrzymuje etykietę mówiącą o tym jaka część mowy reprezentowana jest przez aktualny token.</li>
    <li>Wykrywanie ważnych z naszego punktu widzenia fraz (nazwy produktów, technologii itp.)</li>
    <li>...</li>
</ol>

Mówiąc o encjach nazwanych (Named Entities) - mówimy o frazach, którym nadaliśmy określony typ, np: "01.06.2018" - typ "Data", "Poznań, Polska" - typ "Lokalizacja", "GeForce 1080 GTX Ultra" - typ "Sprzęt Komputerowy".



## Własny NER - trening z użyciem algorytmu CRF (Conditional Random Fields)

Wykrywacze encji wytrenowane są do odnajdywania popularnych typów fraz (Daty, Lokalizacje, Osoby, ...). Co jednak, kiedy chcielibyśmy wykrywać zdefiniowane przez nas typy danych (np. sprzęt komputerowy), które nie są domyśnie wspierane przez istniejące modele? Musielibyśmy wytrenować własnego NERa. Użyjmy paczki 'pycrfsuite' do tego celu.

PyCRFSuite implementuje algorytm CRF - bardzo wydajny algorytm, który potrafi uczyć się tagowania poszczególnych słów z użyciem np. kodowania IOB. Aby rozróżnić różne rodzaje encji, często tagi "I" i "B" kodowania IOB opatruje się dodatkowym sufiksem. Np. B-Date - oznacza początek daty, a I-Location - kontynuację frazy zawierającej lokację.

Ponieważ to czy dane słowo jest encją nazwaną zależy zarówno od tego jak dane słowo wygląda, jak i od słów poprzedzających i następujących po aktualnym - w opisie cech CRFów również uwzględnia się informacje o okalających słowach.

**Zadanie 3 (2 punkty)** Wytrenuj model, który będzie tagował poszczególne słowa w tekście z użyciem pycfrsuite. Aby to zrobić, wykonaj podzadania w krokach poniżej.

Nasz NER będzie się uczyć etykiet na zbiorze tekstów hiszpańskich, które poddane są podziałowi na zdania, tokenizacji, tagowaniem częściami mowy i etykietami encji do wykrycia w formacie IOB. 

In [4]:
!pip install python-crfsuite
import nltk
nltk.download('conll2002')
import sklearn
import pycrfsuite

train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train')) # załaduj korpus treningowy dla języka hiszpańskiego
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))  # załaduj korpus testowy dla języka hiszpańskiego
train_sents[2] # wyświetla przykładowe zdanie, aby zobaczyć jak reprezentowane są dane

     |████████████████████████████████| 747kB 13.9MB/s 
[nltk_data] Downloading package conll2002 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2002.zip.


[('El', 'DA', 'O'),
 ('Abogado', 'NC', 'B-PER'),
 ('General', 'AQ', 'I-PER'),
 ('del', 'SP', 'I-PER'),
 ('Estado', 'NC', 'I-PER'),
 (',', 'Fc', 'O'),
 ('Daryl', 'VMI', 'B-PER'),
 ('Williams', 'NC', 'I-PER'),
 (',', 'Fc', 'O'),
 ('subrayó', 'VMI', 'O'),
 ('hoy', 'RG', 'O'),
 ('la', 'DA', 'O'),
 ('necesidad', 'NC', 'O'),
 ('de', 'SP', 'O'),
 ('tomar', 'VMN', 'O'),
 ('medidas', 'NC', 'O'),
 ('para', 'SP', 'O'),
 ('proteger', 'VMN', 'O'),
 ('al', 'SP', 'O'),
 ('sistema', 'NC', 'O'),
 ('judicial', 'AQ', 'O'),
 ('australiano', 'AQ', 'O'),
 ('frente', 'RG', 'O'),
 ('a', 'SP', 'O'),
 ('una', 'DI', 'O'),
 ('página', 'NC', 'O'),
 ('de', 'SP', 'O'),
 ('internet', 'NC', 'O'),
 ('que', 'PR', 'O'),
 ('imposibilita', 'VMI', 'O'),
 ('el', 'DA', 'O'),
 ('cumplimiento', 'NC', 'O'),
 ('de', 'SP', 'O'),
 ('los', 'DA', 'O'),
 ('principios', 'NC', 'O'),
 ('básicos', 'AQ', 'O'),
 ('de', 'SP', 'O'),
 ('la', 'DA', 'O'),
 ('Ley', 'NC', 'B-MISC'),
 ('.', 'Fp', 'O')]

**Zadanie 3a (1 punkt)** Tworzenie cech. PyCRFSuite oczekuje, że każde słowo opisane będzie zestawem odpowiednich cech w formie pythonowego słownika. Uzupełnij kod funkcji word2features (sekcje TODO) tak, aby stworzyć odpowiednie cechy zgodnie z nazwami i komentarzami do poszczególnych pól.

In [5]:
def word2features(sent, i):
    word = sent[i][0]  # sent[i] ma postać np. ('Ley', 'NC', 'B-MISC'); Indeks 0 oznacza pierwszy element z nawiasów (tupli), czyli w tym przypadku 'Ley'
    postag = sent[i][1] # sent[i] ma postać np. ('Ley', 'NC', 'B-MISC'); Indeks 0 oznacza pierwszy element z nawiasów (tupli), czyli w tym przypadku 'NC'
    
    features = {      # cechy aktualnego słowo
        'bias': 1.0,
        'lowercase_word': word.lower(), # TODO, tutaj słowo małymi literami
        'word_last_3_chars': word[-3:], # TODO, tutaj ostatnie 3 znaki słowa
        'word_last_2_chars': word[-2:], # TODO, tutaj ostatnie 2 znaki słowa
        'word_is_uppercase': word.isupper(), # TODO, tutaj flaga (True/False), czy słowo jest uppercase
        'word_is_digit': word.isdigit(), # TODO, tutaj flaga (True/False), czy słowo jest liczbą
        'postag': postag, # TODO, tutaj pos-tag (patrz początek definicji funkcji)
        'postag_first_two_chars': postag[:2] # TODO, tutaj pierwsze 2 znaki pos-tagu  
    }
    if i > 0:         # jeśli nasze słowo nie jest pierwszym w zdaniu - dodajmy do zbioru naszych cech cechy poprzedniego tokenu
        word1 = sent[i-1][0]    # poprzednie słowo
        postag1 = sent[i-1][1]  # poprzedni pos-tag
        
        features.update({       # funkcja update() na słowniku dopisuje dodatkowe atrybuty do istniejącego słownika
            'previous_word_lower': word1.lower(), # TODO, tutaj poprzednie słowo małymi literami
            'previous_word_is_upppercase': word1.isupper(), # TODO, tutaj flaga (True/False), czy słowo jest uppercase
            'previous_word_postag': postag1, # TODO, tutaj pos-tag poprzedniego słowa 
            'previous_word_postag_first_two_chars': postag1[:2] # TODO, tutaj pierwsze 2 znaki pos-tagu  poprzedniego słowa
        })
    else:
        features['BOS'] = True   # jeśli to pierwszy token - ustawmy cechę BOS (Begin of Sentence) na True
        
    if i < len(sent)-1:          # Jeśli nasze słowo nie jest ostatnim - dodajmy do zbioru cech cechy następnego słowa 
        word1 = sent[i+1][0]     # następne słowo
        postag1 = sent[i+1][1]   # następny postag
        
        features.update({        # funkcja update() na słowniku dopisuje dodatkowe atrybuty do istniejącego słownika
            'next_word_is_lower': word1.islower(), # TODO, tutaj flaga - czy następne słowo małymi literami
            'next_word_is_upppercase': word1.isupper(),  # TODO, tutaj flaga (True/False), czy słowo jest uppercase
            'next_word_postag': postag1, # TODO, tutaj pos-tag następnego słowa 
            'next_word_postag_first_two_chars': postag1[:2] # TODO, tutaj pierwsze 2 znaki pos-tagu  następnego słowa
        })
    else:
        features['EOS'] = True   # jeśli to ostatni token - ustawmy cechę EOS (End of Sentence) na True
                
    return features



def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))] # zamień każde słowo ze zdania na słownik cech

In [6]:
sent2features(train_sents[0])[0]

{'BOS': True,
 'bias': 1.0,
 'lowercase_word': 'melbourne',
 'next_word_is_lower': False,
 'next_word_is_upppercase': False,
 'next_word_postag': 'Fpa',
 'next_word_postag_first_two_chars': 'Fp',
 'postag': 'NP',
 'postag_first_two_chars': 'NP',
 'word_is_digit': False,
 'word_is_uppercase': False,
 'word_last_2_chars': 'ne',
 'word_last_3_chars': 'rne'}

Oczekiwany rezultat: 
<pre>
{'BOS': True,
 'bias': 1.0,
 'lowercase_word': 'melbourne',
 'next_word_lower': 'False',
 'next_word_is_upppercase': False,
 'next_word_postag': 'Fpa',
 'next_word_postag_first_two_chars': 'Fp',
 'postag': 'NP',
 'postag_first_two_chars': 'NP',
 'word_is_digit': False,
 'word_is_uppercase': False,
 'word_last_2_chars': 'ne',
 'word_last_3_chars': 'rne'}
</pre>
 
 **Zadanie 3b (1 punkt) - napisz ciała funkcji pomocniczych, które dla aktualnego zdania z train_sents i test_sents zwrócą:**
 <ul>
     <li>sent2labels - zwróci ciąg oczkiwanych etykiet dla każdego wyrazu. parametr sent jest listą słów, z których każde słowo opisane jest trójką: tekst słowa, pos-tag słowa, etykieta słowa; np. ('Abogado', 'NC', 'B-PER') </li>
     <li>sent2tokens - analogicznie do powyższego, jednak zamiast etykiet zwróci ciąg słów bez pos-tagów i etykiet.</li>
     <li>get_all_labels - funkcja, która ze zbioru wszystkich zdań treningowych wyświetli zbiór etykiet (zbiór, czyli bez powtórzeń). Funkcja pokaże nam ilu etykiet chcemy się nauczyć, aby móc ocenić trudność naszego problemu.</li>
 </ul>

In [7]:
def sent2labels(sent):
    return [s[2] for s in sent]

def sent2tokens(sent):
    return [s[0] for s in sent]

def get_all_labels(train_sents):
    return set([s[2] for sent in train_sents for s in sent])

print(sent2labels(train_sents[0]))
print(sent2tokens(train_sents[0]))
print(get_all_labels(train_sents))

['B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O']
['Melbourne', '(', 'Australia', ')', ',', '25', 'may', '(', 'EFE', ')', '.']
{'I-MISC', 'O', 'B-ORG', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG', 'I-LOC', 'B-MISC'}


Oczekiwany rezultat:
<pre>
['B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O']
['Melbourne', '(', 'Australia', ')', ',', '25', 'may', '(', 'EFE', ')', '.']
{'I-PER', 'I-MISC', 'B-LOC', 'I-LOC', 'B-PER', 'B-MISC', 'I-ORG', 'B-ORG', 'O'}
</pre>

Uruchom poniższy kod i sprawdź czego nauczył się nasz NER.

In [8]:
X_train = [sent2features(s) for s in train_sents] # Stwórz cechy zbioru treningowego
y_train = [sent2labels(s) for s in train_sents]   # Pobierz etykiety zbioru treningowego

X_test = [sent2features(s) for s in test_sents]   # Stwórz cechy zbioru testowego
y_test = [sent2labels(s) for s in test_sents]     # Pobierz etykiety zbioru testowego

trainer = pycrfsuite.Trainer(verbose=False)    # stwórz obiekt trenujący

for xseq, yseq in zip(X_train, y_train):       # iteruj po zdaniach i etykietach
    trainer.append(xseq, yseq)                 # dopisuj do obiektu trenującego nasze dane
    
trainer.set_params({
    'c1': 1.0,   # parametr regularyzacyjny L1
    'c2': 1e-3,  # parametr regularyzacyjny L2
    'max_iterations': 50,  # maksymalna liczba iteracji
    # dodaj tranzycje, które nie są obserwowane ale są możliwe
    'feature.possible_transitions': True
})

trainer.train('conll2002-esp.crfsuite')       # wytrenuj model i zapisz do pliku!

tagger = pycrfsuite.Tagger()                  # stwórz tagger, który będzie nadawał etykiety naszej sekwencji
tagger.open('conll2002-esp.crfsuite')         # załaduj do niego wytrenowany model
example_sent = test_sents[0]                  # weź pierwsze z brzegu zdanie, które nie brało udziału w treningu
print(' '.join(sent2tokens(example_sent)), end='\n\n')   # wyświetl je...

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))  # zobacz, co generuje nasz model
print("Correct:  ", ' '.join(sent2labels(example_sent)))                # i to, czego oczekiwano!

La Coruña , 23 may ( EFECOM ) .

Predicted: B-LOC I-LOC O O O O B-ORG O O
Correct:   B-LOC I-LOC O O O O B-ORG O O


## Named entity recognition za pomocą sieci neuronowych:

Jeśli zastanawiacie się jak zrobić NERa za pomocą sieci neuronowych (Keras), to na Kaggle jest świetny fragment kodu: https://www.kaggle.com/ananysharma/ner-with-bi-lstm

Po ostatnich zajęciach ten kod powinien być prosty do zrozumienia :)

Bi-LSTM to dwukierunkowe LSTMy (zerknijcie na ostatnie slajdy z wprowdzenia do RNN (laboratoria 6))